In [1]:
import polars as pl
import numpy as np
import dsds.metrics as me
import dsds.prescreen as ps
import dsds.sample as sa
import dsds.fs as fs
import dsds.transform as t
import numpy as np 

In [ ]:
df = pl.DataFrame({
    "text1":["abc,ggg", "abc,sss", "ccc,abc"],
    "text2":["aaa,bbb", "ccc,aaa", "bbb,ccc"]
})

In [ ]:
ps.infer_multicategorical(df, separator=",")

In [2]:
df = pl.DataFrame({
    "time": ["2021-01-01", "2021-01-03", "2021-02-01","2021-02-11","2021-03-01","2021-03-02"],
    "a1": [None, 1,2,3,4, None,],
    "a2": [1,2, None,3,4, None,],
})
df = df.with_columns(pl.col("time").str.to_date())
ps.over_time_report(df, cols=["a1", "a2"], time_col="time", metrics=["null", "invalid", "min", "max"])

year,month,a1_null%,a2_null%,a1_invalid%,a2_invalid%,a1_min,a2_min,a1_max,a2_max
i32,u32,f64,f64,f64,f64,i64,i64,i64,i64
2021,1,0.5,0.0,0.5,0.0,1,1,1,2
2021,2,0.0,0.5,0.0,0.5,2,3,3,3
2021,3,0.5,0.5,0.5,0.5,4,4,4,4


In [3]:
df2 = pl.concat([df for _ in range(10000)])
df2.shape

(60000, 3)

In [5]:
%%timeit
for frames in sa.time_window_slide(df2, "time", interval="monthly",length=3):
    first, second, third = frames

2.81 ms ± 53.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
int(3.5)

In [ ]:
df = pl.concat([pl.read_csv("../data/advertising.csv") for _ in range(10)])

In [ ]:
df.group_by("One_Hot_Test").count().collect()

In [ ]:
783 * 0.5

In [ ]:
sa.simple_downsample(df, subgroup=pl.col("One_Hot_Test") == "A", sample_frac=0.5).group_by("One_Hot_Test").count().collect()

In [ ]:
%%timeit
df.select(
    pl.max_horizontal(pl.col("a"), pl.lit(0.1))
)

In [ ]:
%%timeit
df.select(
    pl.col("a").clip_min(0.1)
)

In [ ]:
pl.show_versions()

In [ ]:
t.scale(df, cols=["a","b"], strategy="robust")

In [ ]:
test = pl.DataFrame(
    {
        "a":[["a", None], ["b", None]]
    }
)

test.select(
    pl.col("a").list.set_difference(pl.Series("test", [None]))
)

In [ ]:
df = pl.read_csv("../data/advertising.csv")
df.head()

In [ ]:
predicted = np.random.random(size=(5000,3))
actual = np.round(np.random.random(size=(5000,3))).astype(np.int8)

In [ ]:
me.precision_recall(actual, predicted)

In [ ]:
df = pl.DataFrame({
    "predicted": predicted[:, 1],
    "actual": actual[:, 1]
})

In [ ]:
from sklearn.metrics import roc_auc_score

print(roc_auc_score(actual, predicted, average="weighted"))
print(me.roc_auc(actual, predicted, strategy="balanced"))

In [ ]:
%%timeit
roc_auc_score(actual, predicted)

In [ ]:
%%timeit
me.roc_auc(actual, predicted, strategy="none")

In [ ]:
%%timeit
roc_auc_score(actual, predicted)

In [ ]:
%%timeit
me.roc_auc(actual, predicted)

In [ ]:
import dsds.prescreen as ps
print(ps.corr_table(df, cols=["Age", "Daily Internet Usage"], corr_with=["Clicked on Ad", "Age Band"]))

In [ ]:
df2 = (
    df.lazy().select(["Area Income", "Ad Topic Line", "City", "Clicked on Ad"])
    .drop(["Ad Topic Line"])
).collect()

In [ ]:
df2.show_graph()

In [ ]:
from scipy.fft import fft
a = np.array([1,2,3,1,2,3,1,2,3])
fft(a)

In [ ]:
df = pl.read_csv("../data/train.csv").with_columns(
    pl.lit(1).alias("feature_1"),
    pl.lit(2).alias("feature_2")
)

In [ ]:
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, load_robot_execution_failures

In [ ]:
download_robot_execution_failures()
timeseries, y = load_robot_execution_failures()

In [ ]:
timeseries

In [ ]:
from tsfresh import extract_features
extracted_features = extract_features(timeseries, column_id="id", column_sort="time")

In [ ]:
extracted_features